In [62]:
import pandas as pd
import sys
import os

sys.path.append(os.path.abspath("../../"))
from src.utils.Preprocessing import classify_cancer_type

In [63]:
df_clincal_data = pd.read_csv("../../data/raw/brca_tcga_pub2015_clinical_data.tsv", sep='\t')
print(len(df_clincal_data.index))

print(f"Number of rows: {df_clincal_data.shape[0]} and number of columns: {df_clincal_data.shape[1]}")

818
Number of rows: 818 and number of columns: 110


In [64]:
df_clincal_data["Cancer Type"].unique()

array(['Invasive Breast Carcinoma'], dtype=object)

In [65]:
df_clincal_data["ER Status By IHC"].head(20)

0     Positive
1     Positive
2     Positive
3     Positive
4     Negative
5     Positive
6     Positive
7     Negative
8     Positive
9     Positive
10    Negative
11    Positive
12    Positive
13    Negative
14    Negative
15    Positive
16    Negative
17    Negative
18    Positive
19    Negative
Name: ER Status By IHC, dtype: object

In [66]:
df_clincal_data["PR status by ihc"].head(10)

0    Negative
1    Positive
2    Positive
3    Positive
4    Positive
5    Positive
6    Positive
7    Negative
8    Negative
9    Positive
Name: PR status by ihc, dtype: object

In [67]:
df_clincal_data["HER2 ihc score"].dropna().head(20)

2     1.0
4     2.0
6     2.0
7     0.0
8     3.0
10    0.0
14    2.0
15    2.0
16    2.0
17    1.0
18    2.0
20    3.0
21    1.0
23    1.0
24    0.0
27    2.0
29    2.0
30    2.0
31    2.0
32    1.0
Name: HER2 ihc score, dtype: float64

In [68]:
df_clincal_data["HER2 fish status"].notna()

0      False
1      False
2       True
3      False
4       True
       ...  
813     True
814     True
815     True
816     True
817     True
Name: HER2 fish status, Length: 818, dtype: bool

In [69]:
df_clincal_data["HER2 ihc score"].head(20)

0     NaN
1     NaN
2     1.0
3     NaN
4     2.0
5     NaN
6     2.0
7     0.0
8     3.0
9     NaN
10    0.0
11    NaN
12    NaN
13    NaN
14    2.0
15    2.0
16    2.0
17    1.0
18    2.0
19    NaN
Name: HER2 ihc score, dtype: float64

In [70]:
list_df = classify_cancer_type(df_clinical_data=df_clincal_data)

luminal_A = [x for x in list_df if x == "Luminal A"]
luminal_B = [x for x in list_df if x == "Luminal B"]
HER2_enriched = [x for x in list_df if x == "HER2-enriched"]
TNBC = [x for x in list_df if x == "TNBC"]
UNK = [x for x in list_df if x == "<UNK>"]

print(f"Luminal A: {len(luminal_A)} - Total(%): {len(luminal_A) / len(df_clincal_data):.2f}")
print(f"Luminal B: {len(luminal_B)} - Total(%):{len(luminal_B) / len(df_clincal_data):.2f}")
print(f"HER2-enriched: {len(HER2_enriched)} - Total(%):{len(HER2_enriched) / len(df_clincal_data):.2f}")
print(f"TNBC: {len(TNBC)} - Total(%){len(TNBC) / len(df_clincal_data):.2f} ")
print(f"UNK: {len(UNK)} - Total(%) {len(UNK) / len(df_clincal_data):.2f}")



Luminal A: 330 - Total(%): 0.40
Luminal B: 81 - Total(%):0.10
HER2-enriched: 23 - Total(%):0.03
TNBC: 85 - Total(%)0.10 
UNK: 299 - Total(%) 0.37
